<a href="https://colab.research.google.com/github/chelsea591/Econometria_QLab_PC/blob/main/PC_3/PC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica Calificada 3**

*Integrantes:*
* Lesly Guzmán (20212335)
* Angie Quispe (20231371)
* Chelsea Torres (20200891)

Sobre la base de los datos y lo visto en clases, responda lo siguiente (2 puntos cada una):

#### **1. Realice una descripción de las características de los individuos de la muestra.**

In [1]:
# Instalar e importar las librerías necesarias
!pip install statsmodels pandas linearmodels scipy
!pip install --upgrade linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 6.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

from linearmodels import OLS
from linearmodels.iv import IV2SLS, compare
from statsmodels.api import add_constant
from linearmodels.iv import IVGMM
from scipy import stats

In [3]:
# Explorar el archivo .dta

data = pd.read_stata("card.dta")
data = add_constant(data, has_constant="add") # Agregar una constante que servirá de intercepto

data.head()

,const,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,1.0,2,0,0,7,29,NaN,NaN,158413.0,1,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,1.0,3,0,0,12,27,8.0,8.0,380166.0,1,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,1.0,4,0,0,12,34,14.0,12.0,367470.0,1,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,1.0,5,1,1,11,27,11.0,12.0,380166.0,1,...,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,1.0,6,1,1,12,34,8.0,7.0,367470.0,1,...,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   const     3010 non-null   float64
 1   id        3010 non-null   int16  
 2   nearc2    3010 non-null   int8   
 3   nearc4    3010 non-null   int8   
 4   educ      3010 non-null   int8   
 5   age       3010 non-null   int8   
 6   fatheduc  2320 non-null   float64
 7   motheduc  2657 non-null   float64
 8   weight    3010 non-null   float32
 9   momdad14  3010 non-null   int8   
 10  sinmom14  3010 non-null   int8   
 11  step14    3010 non-null   int8   
 12  reg661    3010 non-null   int8   
 13  reg662    3010 non-null   int8   
 14  reg663    3010 non-null   int8   
 15  reg664    3010 non-null   int8   
 16  reg665    3010 non-null   int8   
 17  reg666    3010 non-null   int8   
 18  reg667    3010 non-null   int8   
 19  reg668    3010 non-null   int8   
 20  reg669    3010 non-null   int8

**Descripción de Variables**

*   **const**: Variable constante añadida para el intercepto en modelos de regresión.
*   **id**: Identificador único para cada individuo.
*   **nearc2**: Indicador si el individuo vivió cerca de un colegio de 2 años (0=no, 1=sí).
*   **nearc4**: Indicador si el individuo vivió cerca de un colegio de 4 años (0=no, 1=sí).
*   **educ**: Años de educación completados.
*   **age**: Edad del individuo.
*   **fatheduc**: Años de educación del padre.
*   **motheduc**: Años de educación de la madre.
*   **weight**: Peso de la encuesta (para ajustes muestrales).
*   **momdad14**: Indicador si ambos padres vivían en casa cuando el individuo tenía 14 años.
*   **sinmom14**: Indicador si solo la madre vivía en casa cuando el individuo tenía 14 años.
*   **step14**: Indicador si un padrastro vivía en casa cuando el individuo tenía 14 años.
*   **reg661** a **reg669**: Variables dummy para la región de residencia en 1966.
*   **south66**: Indicador si el individuo vivía en el sur en 1966.
*   **black**: Indicador si el individuo es afroamericano (0=no, 1=sí).
*   **smsa**: Indicador si el individuo vivía en un Área Estadística Metropolitana (SMSA) en el momento de la encuesta.
*   **south**: Indicador si el individuo vivía en el sur en el momento de la encuesta.
*   **smsa66**: Indicador si el individuo vivía en un SMSA en 1966.
*   **wage**: Salario por hora.
*   **enroll**: Indicador si el individuo estaba matriculado en la escuela en el momento de la encuesta.
*   **KWW**: Puntuación en la prueba de conocimiento del mundo del trabajo (Knowledge of the World of Work).
*   **IQ**: Coeficiente intelectual (IQ).
*   **married**: Indicador si el individuo está casado (0=no, 1=sí).
*   **libcrd14**: Número de libros en la biblioteca de casa cuando el individuo tenía 14 años.
*   **exper**: Experiencia laboral en años (age - educ - 6).
*   **lwage**: Logaritmo natural del salario por hora (`wage`).
*   **expersq**: Experiencia laboral al cuadrado (`exper`^2).

In [5]:
summary = data.describe()
print(summary)

        const           id       nearc2       nearc4         educ  \
count  3010.0  3010.000000  3010.000000  3010.000000  3010.000000   
mean      1.0  2581.748837     0.440864     0.682060    13.263455   
std       0.0  1500.538849     0.496573     0.465753     2.676913   
min       1.0     2.000000     0.000000     0.000000     1.000000   
25%       1.0  1275.500000     0.000000     0.000000    12.000000   
50%       1.0  2541.000000     0.000000     1.000000    13.000000   
75%       1.0  3858.750000     1.000000     1.000000    16.000000   
max       1.0  5225.000000     1.000000     1.000000    18.000000   

               age     fatheduc     motheduc        weight     momdad14  ...  \
count  3010.000000  2320.000000  2657.000000  3.010000e+03  3010.000000  ...   
mean     28.119601    10.003448    10.348137  3.211852e+05     0.789369  ...   
std       3.137004     3.720737     3.179671  1.706459e+05     0.407825  ...   
min      24.000000     0.000000     0.000000  7.560700e+04

In [6]:
# Seleccionar el grupo de variables relevantes en el paper

vars_desc = ["lwage", "educ", "exper", "black", "nearc4"]
data[vars_desc].describe()

,lwage,educ,exper,black,nearc4
count,3010.000000,3010.000000,3010.000000,3010.000000,3010.000000
mean,6.261832,13.263455,8.856146,0.233555,0.682060
std,0.443797,2.676913,4.141672,0.423162,0.465753
min,4.605170,1.000000,0.000000,0.000000,0.000000
25%,5.976985,12.000000,6.000000,0.000000,0.000000
50%,6.286928,13.000000,8.000000,0.000000,1.000000
75%,6.563503,16.000000,11.000000,0.000000,1.000000
max,7.784889,18.000000,23.000000,1.000000,1.000000


**¿Qué información nos provee el promedio de la dependiente en la interpretación de los coeficientes?**

El promedio de la variable dependiente, que en este caso es lwage o logaritmo del salario, es **6.26** (o **577.28** dólares), mientras que el promedio de los años de educación en la muestra es de aproximadamente **13.26** años.

El hecho de que la variable dependiente esté en logaritmos permite que los coeficientes se interpreten como *cambios porcentuales* sobre el salario promedio. Conocer el salario promedio permite traducir esos porcentajes en variaciones monetarias.

**¿Cómo se interpretan los coeficientes en este caso?**

En este estudio la variable dependiente es el logaritmo del salario (lwage). Cuando la variable dependiente está en logaritmos y las variables explicativas están en niveles, el modelo es de tipo log-lineal. En este caso, los coeficientes se interpretan como variaciones porcentuales aproximadas en el salario ante un cambio unitario en la variable explicativa.

En particular, el coeficiente asociado a los años de educación (educ) mide el retorno porcentual a un año adicional de escolaridad. Es decir, indica en cuánto porcentaje aumenta el salario cuando el individuo completa un año adicional de educación, manteniendo constantes las demás variables del modelo.

Por ejemplo, si el coeficiente estimado fuese 0.08, esto implicaría que un año adicional de educación está asociado con un incremento aproximado de 8% en el salario.

#### **2. Regresione usando **OLS****

In [7]:
ols_model = smf.ols(
    'lwage ~ educ + exper + expersq + age + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668',
    data=data
).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     85.48
Date:                Fri, 20 Feb 2026   Prob (F-statistic):          1.74e-218
Time:                        22:11:33   Log-Likelihood:                -1288.8
No. Observations:                3010   AIC:                             2610.
Df Residuals:                    2994   BIC:                             2706.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3400      0.007     51.778      0.0

**¿Qué ocurre con edad? ¿Por qué?**

Según el modelo, el coeficiente de `age` es 0.7332, significativo y positivo. Se espera que age capture la misma variación que `exper` y `educ`. Sin embargo, existe un problema de multicolinealidad muy fuerte, ya que los coeficientes de `exper` y `educ` son negativos, lo cual es económicamente absurdo.

Al quitar `age` del modelo se elimina la relación lineal casi perfecta entre `educ`, `exp` y `age`. Ahora, los coeficientes de educación y experiencia son positivos y tienen una interpretación económica.


In [8]:
# Quitamos edad del modelo
ols_model = smf.ols(
    'lwage ~ educ + exper + expersq + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668',
    data=data
).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     85.48
Date:                Fri, 20 Feb 2026   Prob (F-statistic):          1.74e-218
Time:                        22:11:33   Log-Likelihood:                -1288.8
No. Observations:                3010   AIC:                             2610.
Df Residuals:                    2994   BIC:                             2706.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7394      0.072     66.259      0.0

**Interpretación del estimado obtenido para educación:**

Un año adicional de educación aumenta el salario en aproximadamente 7.47%, manteniendo constantes experiencia, región y demás controles.

**Comparación con la Tabla 2, Columna 2, del paper:**

El coeficiente estimado del modelo es muy similar al reportado en la Tabla 2, Columna 2 del paper, donde el retorno a la educación es 7.4% por año adicional de escolaridad. Esto confirma que la especificación replicada es consistente con los resultados originales del estudio.

####**3. Vea la Tabla 2 de Card (1993).**

**¿Qué se encuentra al inspeccionar el coeficiente estimado de educación? ¿Es creíble este coeficiente? ¿Por qué?**

En la Tabla 2 del paper de David Card (1993), el coeficiente OLS de educación es aproximadamente 0.07. Ese nivel es admisible, pero no necesariamente causal porque la educación puede estar correlacionada con componentes no observables, como la habilidad, que también tienen un efecto sobre el salario. En ese sentido, el estimador OLS es inconsistente:
$$
Cov(educ, u) \neq 0
$$



**¿Qué 2 posibles fuentes de inconsistencia podrían ocurrir como resultado de la omisión de variables?**

**a) Sesgo por habilidad (Ability Bias):** La habilidad es una variable no observada que permite obtener salarios más altos, sin importar el nivel educativo. Si un individuo con habilidad consigue más años de escolaridad e ingresos que el promedio, entonces el sesgo será positivo.

$$
\text{Inconsistencia} = \gamma \frac{\text{Cov}(educ,a)}{\text{Var}(educ)}
$$

* $a$: habilidad cognitiva no observada
* $\gamma > 0$ porque a mayor habilidad, mayor salario
* $\text{Cov}(educ, a) > 0$ porque las personas más hábiles estudian más

**b) Impaciencia:** Algunas personas son impacientes y prefieren integrarse al mercado laboral en lugar de continuar con sus estudios. Esta variable tiene guarda una relación negativa con los años de escolaridad y el salario, ya que una mayor impaciencia implica menos años de escolaridad.

$$
\text{Inconsistencia} = \gamma \frac{\text{Cov}(educ,a)}{\text{Var}(educ)}
$$

* $a$: impaciencia por trabajar
* $\gamma > 0$ porque acumula experiencia laboral y, por tanto, su valor en el mercado
* $\text{Cov}(educ, a) < 0$ porque a más impaciencia, menos educación

Finalmente, el sesgo será negativo.




####**4. Estime la ecuación de primera etapa ¿Qué debemos ver en esta regresión? ¿Qué podría decir respecto de la correlación parcial existente entre educ y nearc4?**

In [9]:
# Estimamos la primera ecuación
y_ols = data['educ']  # Variable endógena (Primera etapa)
X_ols = sm.add_constant(data[['nearc4', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']])  # Instrumento (Primera etapa), Exógenas
model_first_stage = sm.OLS(y_ols, X_ols).fit()
print(model_first_stage.summary())

# F test of excluded instruments:
# Obtener los coeficientes y la matriz de covarianza para el instrumento excluido
coef_nearc4 = model_first_stage.params['nearc4']
cov_nearc4 = model_first_stage.cov_params().loc['nearc4', 'nearc4']
# Calcular el F-statistic para el instrumento excluido
f_stat = (coef_nearc4 ** 2) / cov_nearc4
# Grados de libertad
df_num = 1  # Solo un instrumento excluido (nearc4)
df_den = model_first_stage.df_resid  # Grados de libertad residuales
# Calcular el p-valor asociado usando scipy.stats
p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)
print(f"F test of excluded instruments:\n F-statistic = {f_stat:.2f}\n Prob > F = {p_value:.4f}")

print("Stock-Yogo weak ID test critical values for single endogenous regressor:")
print("                                   10% maximal IV size             16.38")
print("                                   15% maximal IV size              8.96")
print("                                   20% maximal IV size              6.66")
print("                                   25% maximal IV size              5.53")
print("Source: Stock-Yogo (2005).")


                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     182.1
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        22:11:33   Log-Likelihood:                -6258.5
No. Observations:                3010   AIC:                         1.255e+04
Df Residuals:                    2994   BIC:                         1.265e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.8485      0.211     79.805      0.0

####**5. Estime el modelo usando el estimador de Variables Instrumentales, usando nearc4 como un instrumento para la variable educ. Dibuje el DAG correspondiente y explique por qué se espera que esta estrategia funcione. Comente sus resultados y compárelos con los obtenidos en el punto 2.**

In [10]:
# Estimación IV2SLS

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data['nearc4']).fit(cov_type="unadjusted")
print(model_second_stage)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.2382
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2343
No. Observations:                3010   F-statistic:                    769.20
Date:                Fri, Feb 20 2026   P-value (F-stat)                0.0000
Time:                        22:11:33   Distribution:                 chi2(15)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.7740     0.9325     4.0473     0.0001      1.9464      5.6016
exper          0.1083     0.0236     4.5886     0.00

#### **6. Testee la presencia de instrumentos débiles usando:**

* **i) el estadístico F**

In [11]:
# Estimamos la primera ecuación
y_ols = data['educ']  # Variable endógena (Primera etapa)
X_ols = sm.add_constant(data[['nearc4', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']])  # Instrumento (Primera etapa), Exógenas
model_first_stage = sm.OLS(y_ols, X_ols).fit()
print(model_first_stage.summary())

# F test of excluded instruments:
# Obtener los coeficientes y la matriz de covarianza para el instrumento excluido
coef_nearc4 = model_first_stage.params['nearc4']
cov_nearc4 = model_first_stage.cov_params().loc['nearc4', 'nearc4']
# Calcular el F-statistic para el instrumento excluido
f_stat = (coef_nearc4 ** 2) / cov_nearc4
# Grados de libertad
df_num = 1  # Solo un instrumento excluido (nearc4)
df_den = model_first_stage.df_resid  # Grados de libertad residuales
# Calcular el p-valor asociado usando scipy.stats
p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)
print(f"F test of excluded instruments:\n F-statistic = {f_stat:.2f}\n Prob > F = {p_value:.4f}")

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     182.1
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        22:11:33   Log-Likelihood:                -6258.5
No. Observations:                3010   AIC:                         1.255e+04
Df Residuals:                    2994   BIC:                         1.265e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.8485      0.211     79.805      0.0

* **ii) el estadístico de Cragg y Donald (1993)**

In [12]:
# Se usa el siguiente codigo en stata :

#	ivregress 2sls lwage (educ = nearc4) exper expersq black south smsa smsa66 reg661-reg668
#	estat firststage

In [13]:
# Resultado
'''
 Minimum eigenvalue statistic = 13.2558

  Critical Values                      # of endogenous regressors:    1
  Ho: Instruments are weak             # of excluded instruments:     1
  ---------------------------------------------------------------------
                                     |    5%     10%     20%     30%
  2SLS relative bias                 |         (not available)
  -----------------------------------+---------------------------------
                                     |   10%     15%     20%     25%
  2SLS Size of nominal 5% Wald test  |  16.38    8.96    6.66    5.53
  LIML Size of nominal 5% Wald test  |  16.38    8.96    6.66    5.53
  ---------------------------------------------------------------------

'''

'\n Minimum eigenvalue statistic = 13.2558\n\n  Critical Values                      # of endogenous regressors:    1\n  Ho: Instruments are weak             # of excluded instruments:     1\n  ---------------------------------------------------------------------\n                                     |    5%     10%     20%     30%\n  2SLS relative bias                 |         (not available)\n  -----------------------------------+---------------------------------\n                                     |   10%     15%     20%     25%\n  2SLS Size of nominal 5% Wald test  |  16.38    8.96    6.66    5.53\n  LIML Size of nominal 5% Wald test  |  16.38    8.96    6.66    5.53\n  ---------------------------------------------------------------------\n\n'

* **iii) las tablas de Stock y Yogo (2005) con respecto a la medida del test de Wald**

In [14]:
# Calcular el F-statistic para el instrumento excluido
f_stat = (coef_nearc4 ** 2) / cov_nearc4
# Grados de libertad
df_num = 1  # Solo un instrumento excluido (nearc4)
df_den = model_first_stage.df_resid  # Grados de libertad residuales
# Calcular el p-valor asociado usando scipy.stats
p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)
print(f"F test of excluded instruments:\n F-statistic = {f_stat:.2f}\n Prob > F = {p_value:.4f}")

print("Stock-Yogo weak ID test critical values for single endogenous regressor:")
print("                                   10% maximal IV size             16.38")
print("                                   15% maximal IV size              8.96")
print("                                   20% maximal IV size              6.66")
print("                                   25% maximal IV size              5.53")
print("Source: Stock-Yogo (2005).")

F test of excluded instruments:
 F-statistic = 13.26
 Prob > F = 0.0003
Stock-Yogo weak ID test critical values for single endogenous regressor:
                                   10% maximal IV size             16.38
                                   15% maximal IV size              8.96
                                   20% maximal IV size              6.66
                                   25% maximal IV size              5.53
Source: Stock-Yogo (2005).


#### **7. Use ```nearc2``` y ```nearc4``` como instrumentos para ```educ```. Primero estime la primera etapa para ```educ``` y analice cuál de los dos instrumentos está más fuertemente relacionada parcialmente con educ. Después use el estimador de IV usando por separado ```nearc2``` y ```nearc4``` como instrumento para educ. Luego use el de 2SLS (incluyendo ambos instrumentos). Discuta sus resultados.  Verifique que el estimador 2SLS es válido.**


In [15]:
# Estimamos la primera ecuación con los dos instrumentos
y_ols = data['educ']  # Variable endógena (Primera etapa)
X_ols = sm.add_constant(data[['nearc4', 'nearc2','exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']])  # Instrumento (Primera etapa), Exógenas
model_first_stage = sm.OLS(y_ols, X_ols).fit()
print(model_first_stage.summary())



                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     171.0
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        22:11:33   Log-Likelihood:                -6257.2
No. Observations:                3010   AIC:                         1.255e+04
Df Residuals:                    2993   BIC:                         1.265e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.7731      0.216     77.528      0.0

In [16]:
# Regresión de la primera ecuación con nearc2
y_ols_2 = data['educ']  # Variable endógena (Primera etapa)
X_ols_2 = sm.add_constant(data[['nearc2','exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']])  # Instrumento (Primera etapa), Exógenas
model_first_stage_2 = sm.OLS(y_ols_2, X_ols_2).fit()
print(model_first_stage_2.summary())

# F test of excluded instruments:
# Obtener los coeficientes y la matriz de covarianza para el instrumento excluido
coef_nearc2 = model_first_stage_2.params['nearc2']
cov_nearc2 = model_first_stage_2.cov_params().loc['nearc2', 'nearc2']
# Calcular el F-statistic para el instrumento excluido
f_stat_2 = (coef_nearc2 ** 2) / cov_nearc2
# Grados de libertad
df_num_2 = 1  # Solo un instrumento excluido (nearc2)
df_den_2 = model_first_stage_2.df_resid  # Grados de libertad residuales
# Calcular el p-valor asociado usando scipy.stats
p_value = 1 - stats.f.cdf(f_stat_2, df_num_2, df_den_2)
print(f"F test of excluded instruments:\n F-statistic = {f_stat_2:.2f}\n Prob > F = {p_value:.4f}")


                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                     180.8
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        22:11:33   Log-Likelihood:                -6263.9
No. Observations:                3010   AIC:                         1.256e+04
Df Residuals:                    2994   BIC:                         1.266e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.9192      0.213     79.415      0.0

In [17]:
# Regresión de la primera ecuación con nearc4
y_ols_4 = data['educ']  # Variable endógena (Primera etapa)
X_ols_4 = sm.add_constant(data[['nearc4','exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']])  # Instrumento (Primera etapa), Exógenas
model_first_stage_4 = sm.OLS(y_ols_4, X_ols_4).fit()
print(model_first_stage_4.summary())

# F test of excluded instruments:
# Obtener los coeficientes y la matriz de covarianza para el instrumento excluido
coef_nearc4 = model_first_stage_4.params['nearc4']
cov_nearc4 = model_first_stage_4.cov_params().loc['nearc4', 'nearc4']
# Calcular el F-statistic para el instrumento excluido
f_stat_4 = (coef_nearc4 ** 2) / cov_nearc4
# Grados de libertad
df_num_4 = 1  # Solo un instrumento excluido (nearc4)
df_den_4 = model_first_stage_4.df_resid  # Grados de libertad residuales
# Calcular el p-valor asociado usando scipy.stats
p_value = 1 - stats.f.cdf(f_stat_4, df_num_4, df_den_4)
print(f"F test of excluded instruments:\n F-statistic = {f_stat_4:.2f}\n Prob > F = {p_value:.4f}")


                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     182.1
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        22:11:33   Log-Likelihood:                -6258.5
No. Observations:                3010   AIC:                         1.255e+04
Df Residuals:                    2994   BIC:                         1.265e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.8485      0.211     79.805      0.0

In [18]:
# Estimación IV2SLS

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data[['nearc4', 'nearc2']]).fit(cov_type="unadjusted")
print(model_second_stage)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1702
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1660
No. Observations:                3010   F-statistic:                    709.89
Date:                Fri, Feb 20 2026   P-value (F-stat)                0.0000
Time:                        22:11:33   Distribution:                 chi2(15)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.3397     0.8922     3.7434     0.0002      1.5911      5.0883
exper          0.1188     0.0227     5.2237     0.00

In [19]:
# --- Pruebas ---
# 1. Definimos los nombres de los instrumentos que queremos testear
instrumentos = ['nearc4', 'nearc2']

# 2. Creamos la hipótesis: "nearc4 = 0" y "nearc2 = 0"
# Esto le dice a statsmodels que pruebe si ambos son cero al mismo tiempo
hypotheses = '(nearc4 = 0), (nearc2 = 0)'

# 3. Realizamos el test de Wald (que en OLS nos da el estadístico F)
f_test = model_first_stage.f_test(hypotheses)

# 4. Imprimimos el resultado con el formato que ya tenías
print(f"F test of excluded instruments (nearc4 y nearc2):")
print(f" F-statistic = {f_test.fvalue:.4f}")
print(f" Prob > F = {f_test.pvalue:.4f}")

F test of excluded instruments (nearc4 y nearc2):
 F-statistic = 7.8931
 Prob > F = 0.0004


#### **8. Hemos asumido hasta ahora homoscedasticidad ¿Es razonable esto?¿Qué permite el estimador de GMM que no permite el de IV en 2 etapas bajo heterosk. Compare los resultados (del coeficiente educación) del estimador 2SLS con el del estimador GMM bajo heterocedasticidad**

In [21]:
# Estimación IV2SLS asumiendo heterocedasticidad

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data[['nearc4', 'nearc2']]).fit(cov_type="robust")
print(model_second_stage)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1702
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1660
No. Observations:                3010   F-statistic:                    774.72
Date:                Fri, Feb 20 2026   P-value (F-stat)                0.0000
Time:                        22:12:32   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.3397     0.8909     3.7486     0.0002      1.5935      5.0859
exper          0.1188     0.0229     5.1906     0.00

In [22]:
# Ajustar el modelo GMM ('dependent ~ exog + [endog ~ instruments]')
model_second_stage_GMM = IVGMM.from_formula('lwage ~ const + exper + expersq + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668 + [educ ~ nearc4 + nearc2]',
                                        data).fit(cov_type="robust")
print(model_second_stage_GMM)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.1760
Estimator:                     IV-GMM   Adj. R-squared:                 0.1718
No. Observations:                3010   F-statistic:                    781.32
Date:                Fri, Feb 20 2026   P-value (F-stat)                0.0000
Time:                        22:12:40   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.3721     0.8873     3.8004     0.0001      1.6330      5.1112
exper          0.1180     0.0228     5.1747     0.00

#### **9. Sigamos con el caso donde se tienen dos instrumentos. Dado lo hallado en términos de si los instrumentos por separado son débiles o no en la pregunta 7, ¿qué concluye ud. a partir del resultado del test de sobreidentificación? ¿Con qué instrumento o instrumentos se quedaría al final? Realícelo asumiendo heterocedascidad para el estimador GMM y 2SLS (HINT: piense 2 veces)**


In [28]:
# Estimación IV2SLS

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data[['nearc4', 'nearc2']]).fit(cov_type="robust")
print(model_second_stage)

# Prueba de sobreidentificación (Sargan)
print("\nModelo IV2SLS")
sargan_test = model_second_stage.sargan
print("\nPrueba de Sobreidentificación (Sargan):")
print(sargan_test)

# Prueba de sobreidentificación (Basmann)
basmann_test = model_second_stage.basmann
print("\nPrueba de Sobreidentificación (Basmann):")
print(basmann_test)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1702
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1660
No. Observations:                3010   F-statistic:                    774.72
Date:                Fri, Feb 20 2026   P-value (F-stat)                0.0000
Time:                        23:06:26   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.3397     0.8909     3.7486     0.0002      1.5935      5.0859
exper          0.1188     0.0229     5.1906     0.00

In [27]:
# En GMM

print("\nModelo GMM")
hansen_j_test = model_second_stage_GMM._j_stat
print("\nPrueba de Sobreidentificación (J Hansen):")
print(hansen_j_test)


Modelo GMM

Prueba de Sobreidentificación (J Hansen):
H0: Expected moment conditions are equal to 0
Statistic: 1.2689
P-value: 0.2600
Distributed: chi2(1)


#### **10. ¿Deberíamos usar MCO y aceptar la inconsistencia o usar en vez IV a pesar de que es un estimador consistente pero ineficiente? ¿Hay forma de testear esto? Si sí, Llévela a cabo para el modelo sobre-identificado.  ¿Qué concluye?**

In [31]:
# Estimación IV2SLS con solo 2 instrumento (nearc4 y nearc2)

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data[['nearc4', 'nearc2']]).fit(cov_type="robust")
print(model_second_stage)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1702
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1660
No. Observations:                3010   F-statistic:                    774.72
Date:                Sat, Feb 21 2026   P-value (F-stat)                0.0000
Time:                        00:15:13   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.3397     0.8909     3.7486     0.0002      1.5935      5.0859
exper          0.1188     0.0229     5.1906     0.00

In [34]:
print("\nPruebas con 2 instrumentos:")

endog_test = model_second_stage.wu_hausman()
print("\nPrueba de Endogeneidad (Hausman):")
print(endog_test)

durbin_test = model_second_stage.durbin()
print("\nPrueba de Endogeneidad (Durbin):")
print(durbin_test)


Pruebas con 2 instrumentos:

Prueba de Endogeneidad (Hausman):
Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 3.0568
P-value: 0.0805
Distributed: F(1,2993)

Prueba de Endogeneidad (Durbin):
Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 3.0710
P-value: 0.0797
Distributed: chi2(1)


In [36]:
# Estimación IV2SLS con solo 1 instrumento (nearc4)

model_second_stage = IV2SLS(data['lwage'],
                            data[['const', 'exper', 'expersq', 'black', 'south', 'smsa', 'smsa66', 'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668']],
                            data['educ'],
                            data['nearc4']).fit(cov_type="robust")
print(model_second_stage)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.2382
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2343
No. Observations:                3010   F-statistic:                    840.83
Date:                Sat, Feb 21 2026   P-value (F-stat)                0.0000
Time:                        00:18:46   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.7740     0.9174     4.1137     0.0000      1.9759      5.5720
exper          0.1083     0.0233     4.6376     0.00

In [37]:
print("\nPruebas con 1 instrumento (nearc4):")

endog_test = model_second_stage.wu_hausman()
print("\nPrueba de Endogeneidad (Hausman):")
print(endog_test)

durbin_test = model_second_stage.durbin()
print("\nPrueba de Endogeneidad (Durbin):")
print(durbin_test)


Pruebas con 1 instrumento (nearc4):

Prueba de Endogeneidad (Hausman):
Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 1.1676
P-value: 0.2800
Distributed: F(1,2993)

Prueba de Endogeneidad (Durbin):
Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 1.1738
P-value: 0.2786
Distributed: chi2(1)
